# ATM withdrawal program with the blend of Python PostgreSQL with trigger function

#Requirement:

The task is to create a Python program that will withdraw money from a bank account by accepting the Debit Card number, CVV, Expiry Month, Expiry Year, and Amount to be withdrawn. The program will then check whether the Debit Card is valid and has not expired, and whether the amount to be withdrawn is less than or equal to the available balance in the account. If the conditions are satisfied, the program will deduct the amount from the respective account and store the after update changes of the hdfc table in the hdfc_transaction_logs. The bank table will have Account Number, Name, Address, Balance, Phone, Email, Branch Code, and Security Code. The Debit Card Table will have Debit Card Number, Account Number, CVV, Name, Expiry Month, and Expiry Year.

# Creating tables ,Trigger function and firing Trigger on HDFC(desired table)

In [ ]:
# Create table named HDFC with required columns and sample values
#sample values are given for example
"""CREATE TABLE HDFC (
  account_no INT PRIMARY KEY,
  name VARCHAR(50) NOT NULL,
  addresses VARCHAR(255),
  balance NUMERIC(15, 2) NOT NULL,
  phone VARCHAR(15),
  email_id VARCHAR(50),
  branch_code VARCHAR(10),
  security_code VARCHAR(10)
);"""

"""INSERT INTO HDFC (account_no, name, addresses, balance, phone, email_id, branch_code, security_code)
VALUES (1001, 'John Doe', '123 Main St, Anytown, USA', 10000, '555-1234', 'johndoe@example.com', 'BR100', 'SEC123'),
       (1002, 'Jane Doe', '456 Elm St, Anytown, USA', 20000, '555-5678', 'janedoe@example.com', 'BR200', 'SEC456');"""

#Create table named debit with required columns and sample values

"""CREATE TABLE debit (
  debit_card_no VARCHAR(20) PRIMARY KEY,
  account_no INT NOT NULL,
  cvv VARCHAR(4) NOT NULL,
  name VARCHAR(50) NOT NULL,
  expiry_year INT NOT NULL,
  expiry_month INT NOT NULL
);

INSERT INTO debit (debit_card_no, account_no, cvv, name, expiry_year, expiry_month)
VALUES ('1234567890123456', 1001, '123', 'John Doe', 2025, 12),
       ('2345678901234567', 1002, '456', 'Jane Doe', 2026, 6);"""

#Create table named HDFC_transaction_logs to store transaction logs with required columns


"""CREATE TABLE HDFC_transaction_logs (
  transaction_time TIMESTAMP NOT NULL DEFAULT NOW(),
  account_no INT NOT NULL,
  transaction_type VARCHAR(25) NOT NULL,
  transaction_id SERIAL PRIMARY KEY
);"""

#Create trigger function named withdrawal_logs to perform the desired operation on update of HDFC table

"""CREATE FUNCTION withdrawal_logs() RETURNS TRIGGER AS $$
BEGIN
  INSERT INTO HDFC_transaction_logs (transaction_time, account_no, transaction_type, withdrawal_amount)
  VALUES (NOW(), NEW.account_no, 'withdrawal', OLD.balance - NEW.balance);
  RETURN NEW;
END;
$$ LANGUAGE plpgsql;"""

#Create trigger named hdfc_log to call the trigger function withdrawal_logs() after update of HDFC table

"""CREATE TRIGGER hdfc_log AFTER UPDATE ON HDFC
FOR EACH ROW EXECUTE PROCEDURE withdrawal_logs();"""

# Python program to get inputs and do the data manipulation in the PostgreSQL tables.

In [ ]:
# Importing the psycopg2 library which is used for connecting to PostgreSQL databases
import psycopg2

# This loop will keep running until the user provides valid inputs for all the required information
while True:
    try:
        # Attempt to connect to the database using PostgreSQL with the  credentials. 
        x = psycopg2.connect(user='postgres', password='******', host='localhost', port=5432, database='sakthi')
        
        # Create a cursor object to execute SQL queries
        y = x.cursor()

        # This loop will keep running until the user provides a valid debit card number
        while True:
            try:
                # Ask the user to input their debit card number
                card_no = int(input('Enter the card number: '))
                # Check if the entered debit card number exists in the 'debit' table
                y.execute('SELECT debit_card_no FROM debit WHERE debit_card_no=%s;', [card_no])
                debit_no = y.fetchone()
                if not debit_no:
                    raise ValueError('Invalid debit number')
                # Break out of this loop and continue to the next one if the entered debit card number is valid
                break
            except ValueError:
                print('Invalid card number. Please enter the correct card number.')

        # This loop will keep running until the user provides a valid CVV number
        while True:
            try:
                # Ask the user to input their CVV number
                CVV = int(input('Enter the CVV number: '))
                # Check if the entered CVV number matches the one in the 'debit' table for the entered debit card number
                y.execute('SELECT cvv FROM debit WHERE cvv=%s AND debit_card_no=%s;', [CVV, card_no])
                cvv_no = y.fetchone()
                if not cvv_no:
                    raise ValueError('Invalid CVV number')
                # Break out of this loop and continue to the next one if the entered CVV number is valid
                break
            except ValueError:
                print('Invalid CVV number. Please enter the correct CVV number.')

        # This loop will keep running until the user provides a valid expiry year
        while True:
            try:
                # Ask the user to input the expiry year of their debit card
                Exp_y = int(input('Enter the expiry year: '))
                # Check if the entered expiry year matches the one in the 'debit' table for the entered debit card number and CVV number
                y.execute('SELECT expiry_year FROM debit WHERE expiry_year=%s AND cvv=%s AND debit_card_no=%s;', [Exp_y, CVV, card_no])
                year = y.fetchone()
                if not year:
                    raise ValueError('Invalid expiry year')
                # Break out of this loop and continue to the next one if the entered expiry year is valid
                break
            except ValueError:
                print('Invalid expiry year. Please enter the correct expiry year.')

        # This loop will keep running until the user provides a valid expiry month
        while True:
            try:
                # Ask the user to input the expiry month of their debit card
                Exp_m = int(input('Enter the expiry month: '))
                # Check if the entered expiry month matches the one in the 'debit' table for the entered debit card number, CVV number, and expiry year
                y.execute('SELECT expiry_month FROM debit WHERE expiry_month=%s AND expiry_year=%s AND cvv=%s AND debit_card_no=%s;', [Exp_m, Exp_y, CVV, card_no])
                month = y.fetchone()
                if not month:
                    raise ValueError('Invalid expiry month')
                # Break out of this loop and continue to the next one if the entered expiry month is valid
                break
            except ValueError:
                print('Invalid expiry month. Please enter the correct expiry month.')
        # Execute the SQL query to fetch the account number associated with the given debit card details        
        y.execute('SELECT account_no FROM debit WHERE expiry_month=%s AND expiry_year=%s AND cvv=%s AND debit_card_no=%s;', [Exp_m, Exp_y, CVV, card_no])
        account = y.fetchone()
        # Loop until a valid security code is entered
        while True:
            try:
                # Get the security code from the user and execute the SQL query to check if it's valid
                security = int(input('Enter the security code: '))
                y.execute('SELECT security_code FROM hdfc WHERE account_no=%s AND security_code=%s;', [account, security])
                key = y.fetchone()
                # If the security code is invalid, raise a ValueError
                if not key:
                    raise ValueError('Invalid security number')
                # If the security code is valid, break out of the loop
                break
            except ValueError:
                print('Invalid security number. Please enter the correct security number.')
        # Loop until a valid amount is entered
        while True:
            try:
                # Get the amount to withdraw from the user and execute the SQL query to check the account balance
                amount = int(input('Enter the amount: '))
                y.execute('SELECT balance FROM hdfc WHERE account_no=%s;', [account])
                balance = y.fetchone()
                # If the amount is negative, raise a ValueError
                if amount < 0:
                    raise ValueError('Invalid amount')
                 # If the amount is valid, break out of the loop
                break
            except ValueError:
                print('Invalid amount. Please enter the valid amount.')
        #Validata if the amount is greater than the balance.If the amount is greater than the balance,break out of loop and break out of program
        while True:    
            if amount > balance[0]:
                print('Insufficient balance.')
            break
        break
        # Execute the SQL query to update the account balance and print a success message
        y.execute('UPDATE hdfc SET balance=balance-%s WHERE account_no=%s AND security_code=%s;', [amount, account, security])
        print('You have withdrawn successfully.')
        # Commit the changes to the database and close the connections
        x.commit()
        x.close()
        y.close()
        break
    # Catch any database connection errors
    except psycopg2.Error:       
        print('Failed to connect to the database. Please check the connection.')
        
